## Position using DeepLabCut from Scratch

**Note: make a copy of this notebook and run the copy to avoid git conflicts in the future**

This is a tutorial on how to extract position via DeepLabCut (DLC) using the Spyglass pipeline used in Loren Frank's lab, UCSF. It will walk through creating your DLC project, extracting and labeling frames, training your model, executing pose estimation on a novel behavioral video, processing the pose estimation output to extract a centroid and orientation, and inserting the resulting information into the `PositionOutput` table.<br>
-> This tutorial assumes you've completed [tutorial 0](0_intro.ipynb)<br>
**Note 2: Make sure you are running this within the spyglass-position Conda environment (instructions for install are in the environment_position.yml)**

In [ ]:
from pathlib import Path, PosixPath, PurePath
import os
import glob
import numpy as np
import pandas as pd
import pynwb
import datajoint as dj
import spyglass.common as sgc
import spyglass.position.v1 as sgp
from spyglass.position import PositionOutput

#### Here is a schematic showing the tables used in this notebook.<br>
![dlc_scratch.png|2000x900](./../notebook-images/dlc_scratch.png)

### Table of Contents<a id='TableOfContents'></a>
[`DLCProject`](#DLCProject1)<br>
[`DLCModelTraining`](#DLCModelTraining1)<br>
[`DLCModel`](#DLCModel1)<br>
[`DLCPoseEstimation`](#DLCPoseEstimation1)<br>
[`DLCSmoothInterp`](#DLCSmoothInterp1)<br>
[`DLCCentroid`](#DLCCentroid1)<br>
[`DLCOrientation`](#DLCOrientation1)<br>
[`DLCPosV1`](#DLCPosV1-1)<br>
[`DLCPosVideo`](#DLCPosVideo1)<br>
[`PositionOutput`](#PositionOutput1)<br>

#### [DLCProject](#TableOfContents) <a id="DLCProject1"></a>
__You can click on any header to return to the Table of Contents__

<div class="alert alert-block alert-info">
    <b>Note:</b> The cells within the <code>DLCProject</code> step need to be performed in a local Jupyter notebook to allow for use of the frame labeling GUI.</div>

Let us begin with visualizing the contents of the BodyPart table. This table will store standard names of body parts used within DLC models throughout the lab with a concise description.<br>
><div class="alert alert-block alert-warning">Please do not add to this table unless necessary.</div>

In [ ]:
sgp.BodyPart()

<div class="alert alert-block alert-info">
If the bodyparts you plan to use in your model are not yet in the table, here is code to add bodyparts:
</div>

>```python
sgp.BodyPart.insert([{'bodypart': 'bodypart_1', 'bodypart_description': 'concise description of bodypart'},
                     {'bodypart': 'bodypart_2', 'bodypart_description': 'concise description of bodypart'},],
                    skip_duplicates=True)
```

Next we want to construct a list of videos from which we will extract frames to train the model.<br>The list can either contain dictionaries identifying behavioral videos for NWB files that have already been added to Spyglass, or absolute file paths to the videos you want to use.<br>For this tutorial, we'll use two videos for which we already have frames labeled.

In [ ]:
video_list = [
    {"nwb_file_name": "J1620210529_.nwb", "epoch": 2},
    {"nwb_file_name": "peanut20201103_.nwb", "epoch": 4},
]

Before creating our project, we need to define a few variables.
>First, we want to set a team name to one that exists in the `LabTeam` table to ensure proper permission are set.<br>In this case we'll use "LorenLab", as all Frank Lab members are a part of this team.<br>
We also need to define a `project_name`, which should be a unique identifier for this project. For the tutorial we'll set it as __"tutorial_scratch_yourinitials"__<br>Next, we need to define a list of `bodyparts` for which we want to extract position. The pre-labeled frames we're using include the bodyparts listed below, but please modify as needed for your own project.<br>We also want to define how many frames we want to extract and eventually label from each video we're using. I will typically use 200 `frames_per_video`, but we'll keep it to 100 for efficiency's sake.

In [ ]:
team_name = "LorenLab"
project_name = "tutorial_scratch_DG"
frames_per_video = 100
bodyparts = ["redLED_C", "greenLED", "redLED_L", "redLED_R", "tailBase"]
project_key = sgp.DLCProject.insert_new_project(
    project_name=project_name,
    bodyparts=bodyparts,
    lab_team=team_name,
    frames_per_video=frames_per_video,
    video_list=video_list,
    skip_duplicates=True,
)

Now that we've intialized our project we'll need to extract and label frames.<br>While this has already been done for this tutorial, here are the commands in order to pull up the DLC GUI to perform these actions:
>```python
sgp.DLCProject().run_extract_frames(project_key)
sgp.DLCProject().run_label_frames(project_key)
```

Typically, in order to use pre-labeled frames to your project you'll need to change the values in the labeled-data files. You can do that using the `import_labeled_frames` method. 
<blockquote>This function expects the `project_key` from your new project<br>The absolute path to the project you want to import the labeled frames from<br>The filename (without file extension) of the videos from which you want the frames.
</blockquote>Here we'll use the path to a pre-existing project from tutorial 06

In [ ]:
sgp.DLCProject.import_labeled_frames(
    project_key.copy(),
    import_project_path="/nimbus/deeplabcut/projects/tutorial_model-LorenLab-2022-07-15/",
    video_filenames=["20201103_peanut_04_r2", "20210529_J16_02_r1"],
    skip_duplicates=True,
)

Now we can check the `DLCProject.File` part table and see all of our training files and videos there!

In [ ]:
sgp.DLCProject.File & project_key

<div class="alert alert-block alert-warning">
    <b>This step and beyond should be run on a GPU-enabled machine.</b>
</div>

#### [DLCModelTraining](#ToC)<a id='DLCModelTraining1'></a> 
Please make sure you're running this notebook on a GPU-enabled machine.<br>
Now that we've imported existing frames, we can get ready to train our model.<br>
First, we'll need to define a set of parameters for `DLCModelTrainingParams`, which will get used by DeepLabCut during training<br>
Let's start with `gputouse`<br>

<div class="alert alert-block alert-info">
<code>gputouse</code> determines which GPU core to use for pose estimation. Run the cell below to determine which core has space and set the <code>gputouse</code> variable accordingly.
</div>

In [ ]:
sgp.dlc_utils.get_gpu_memory()

<div class="alert alert-block alert-warning">
Set GPU core here</div>

In [ ]:
gputouse = 3  ## 1-9

Now let's define the rest of our parameters and insert the entry.<br>
(If you want to see all possible parameters that you can pass, checkout the line below):
>```python
sgp.DLCModelTrainingParams.get_accepted_params()
```

In [ ]:
training_params_name = "tutorial_DG"
sgp.DLCModelTrainingParams.insert_new_params(
    paramset_name=training_params_name,
    params={
        "trainingsetindex": 0,
        "shuffle": 1,
        "gputouse": gputouse,
        "net_type": "resnet_50",
        "augmenter_type": "imgaug",
    },
    skip_duplicates=True,
)

Next we'll modify the `project_key` from above to include the necessary entries for `DLCModelTraining`

In [ ]:
if "config_path" in project_key:
    del project_key["config_path"]

And here we can insert an entry into `DLCModelTrainingSelection` and populate `DLCModelTraining` with that entry, which will run training for us.<br>
**Note**: You can stop training at any point using `I + I` or interrupt the Kernel

In [ ]:
sgp.DLCModelTrainingSelection().insert1(
    {
        **project_key,
        "dlc_training_params_name": training_params_name,
        "training_id": 0,
        "model_prefix": "",
    }
)
model_training_key = (
    sgp.DLCModelTrainingSelection
    & {
        **project_key,
        "dlc_training_params_name": training_params_name,
    }
).fetch1("KEY")
sgp.DLCModelTraining.populate(model_training_key)

Here we'll make sure that the entry made it into the table properly!

In [ ]:
sgp.DLCModelTraining() & model_training_key

Populating `DLCModelTraining` automatically inserts the entry into `DLCModelSource`.  `DLCModelSource` is a table that is used to switch between the models we train using Spyglass and pre-existing projects.

In [ ]:
sgp.DLCModelSource() & model_training_key

Notice the `source` field in the table above. It will only accept _"FromImport"_ or _"FromUpstream"_ as entries. Let's checkout the `FromUpstream` part table attached to `DLCModelSource` below.

In [ ]:
sgp.DLCModelSource.FromUpstream() & model_training_key

#### [DLCModel](#TableOfContents) <a id='DLCModel1'></a>
Next we'll get ready to populate the `DLCModel` table, which holds all the relevant information for both pre-trained models and models trained within Spyglass.<br>First we'll need to determine a set of parameters for our model to select the correct model file.<br>We can visualize a default set below:

In [ ]:
sgp.DLCModelParams.get_default()

> Here is the syntax to add your own parameter set:
>```python
dlc_model_params_name = "make_this_yours"
params = {
            "params": {},
            "shuffle": 1,
            "trainingsetindex": 0,
            "model_prefix": "",
        }
sgp.DLCModelParams.insert1({"dlc_model_params_name": dlc_model_params_name, "params": params}, skip_duplicates=True)
```

Now that we've defined a set of parameters and inserted into `DLCModelParams`, we can insert an entry into `DLCModelSelection` and populate `DLCModel`.

In [ ]:
temp_model_key = (sgp.DLCModelSource & model_training_key).fetch1("KEY")
sgp.DLCModelSelection().insert1({
    **temp_model_key,
    "dlc_model_params_name": "default"},
    skip_duplicates=True)
model_key = (sgp.DLCModelSelection & ).fetch1("KEY")
sgp.DLCModel.populate(model_key)

Again, let's make sure that everything looks correct in `DLCModel`.

In [ ]:
sgp.DLCModel() & model_key

#### [DLCPoseEstimation](#TableOfContents) <a id='DLCPoseEstimation1'></a>

Alright, now that we've trained model and populated the `DLCModel` table, we're ready to set-up Pose Estimation on a behavioral video of your choice.<br>For this tutorial, you can choose to use an epoch of your choice, we can also use the one specified below. If you'd like to use your own video, just specify the `nwb_file_name` and `epoch` number and make sure it's in the `VideoFile` table!

In [ ]:
nwb_file_name = "J1620210604_.nwb"
epoch = 14

In [ ]:
sgc.VideoFile() & {"nwb_file_name": nwb_file_name, "epoch": epoch}

To set up pose estimation, we need to make sure a few things are in order. Using `insert_estimation_task` will take care of these steps for us!<br>Briefly, it will convert out video to be in .mp4 format (DLC struggles with .h264) and determine the directory in which we'll store the pose estimation results.<br>
>**`task_mode`** determines whether or not populating `DLCPoseEstimation` runs a new pose estimation, or loads an existing. Use _'trigger'_ unless you've already run this specific pose estimation.<br>**`video_file_num`** will be 0 in almost all cases.<br>__`gputouse`__ has already been set above during the training step. It may be a good idea to make sure that core is still free before moving forward.

In [ ]:
pose_estimation_key = sgp.DLCPoseEstimationSelection.insert_estimation_task(
    {
        "nwb_file_name": nwb_file_name,
        "epoch": epoch,
        "video_file_num": 0,
        **model_key,
    },
    task_mode="trigger",
    params={"gputouse": gputouse, "videotype": "mp4"},
)

And now we populate `DLCPoseEstimation`! This might take a bit...

In [ ]:
sgp.DLCPoseEstimation().populate(pose_estimation_key)

Let's visualize the output from Pose Estimation

In [ ]:
(sgp.DLCPoseEstimation() & pose_estimation_key).fetch_dataframe()

#### [DLCSmoothInterp](#TableOfContents) <a id='DLCSmoothInterp1'></a>

Now that we've completed pose estimation, it's time to identify NaNs and optionally interpolate over low likelihood periods and smooth the resulting positions.<br>First we need to define some parameters for smoothing and interpolation. We can see the default parameter set below.<br>__Note__: it is recommended to use the `just_nan` parameters here and save interpolation and smoothing for the centroid step as this provides for a better end result.

In [ ]:
# The default parameter set to interpolate and smooth over each LED individually
print(sgp.DLCSmoothInterpParams.get_default())

In [ ]:
# The just_nan parameter set that identifies NaN indices and leaves smoothing and interpolation to the centroid step
print(sgp.DLCSmoothInterpParams.get_nan_params())
si_params_name = "just_nan"

> If you'd like to change any of these parameters, here is the syntax to do that
>```python
si_params_name = 'your_unique_param_name'
params = {
    "smoothing_params": {
        "smoothing_duration": 0.##,
        "smooth_method": "moving_avg",
    },
    "interp_params": {
        "likelihood_thresh": 0.##,
    },
    "max_plausible_speed": ###,
    "speed_smoothing_std_dev": 0.###,
}
sgp.DLCSmoothInterpParams().insert1(
    {
        'dlc_si_params_name': si_params_name,
        "params": params,
    },
    skip_duplicates=True)
```

Here we'll create a dictionary with the correct set of keys for the `DLCSmoothInterpSelection` table

In [ ]:
si_key = pose_estimation_key.copy()
fields = list(sgp.DLCSmoothInterpSelection.fetch().dtype.fields.keys())
si_key = {key: val for key, val in si_key.items() if key in fields}
si_key

And now we can insert all of the bodyparts we want to process into `DLCSmoothInterpSelection`<br>
First lets visualize the bodyparts we have available to us.<br>

In [ ]:
print((sgp.DLCPoseEstimation.BodyPart & pose_estimation_key).fetch("bodypart"))

We can use `insert1` to insert a single bodypart, but would suggest using `insert` to insert a list of keys with different bodyparts.

>_Syntax to insert a single bodypart_
>```python
sgp.DLCSmoothInterpSelection.insert1(
    {
        **si_key,
        'bodypart': 'greenLED',
        'dlc_si_params_name': si_params_name,
    },
    skip_duplicates=True)
```

Lets set a list of bodyparts we want to insert and then insert them into `DLCSmoothInterpSelection`.

In [ ]:
bodyparts = ["greenLED", "redLED_C"]
sgp.DLCSmoothInterpSelection.insert(
    [
        {
            **si_key,
            "bodypart": bodypart,
            "dlc_si_params_name": si_params_name,
        }
        for bodypart in bodyparts
    ],
    skip_duplicates=True,
)

And to make sure that all of the bodyparts we want made it into the the selection table, we can visualize the table below.

In [ ]:
sgp.DLCSmoothInterpSelection() & si_key

Now we can populate `DLCSmoothInterp`, which will perform smoothing and interpolation on all of the bodyparts we specified.<br>We can limit the populate using `si_key` since it is bodypart agnostic.

In [ ]:
sgp.DLCSmoothInterp().populate(si_key)

And let's visualize the resulting position data using a scatter plot

In [ ]:
(
    sgp.DLCSmoothInterp() & {**si_key, "bodypart": bodyparts[0]}
).fetch1_dataframe().plot.scatter(x="x", y="y", s=1, figsize=(5, 5))

#### [DLCSmoothInterpCohort](#TableOfContents) <a id='DLCSmoothInterpCohort1'></a>

Now that we've smoothed and interpolated our position data for each bodypart, we need to form a set of bodyparts from which we want to derive a centroid and orientation (or potentially a second set for orientation). This is the goal of the `DLCSmoothInterpCohort` table.

First, let's make a key that represents the 'cohort' we want to form.
> We'll set the `dlc_si_cohort_selection_name` to a concise name<br>We'll also form a dictionary with the bodypart name as the key and the smoothing/interpolation parameter name used for that bodypart as the value.

In [ ]:
cohort_key = si_key.copy()
if "bodypart" in cohort_key:
    del cohort_key["bodypart"]
if "dlc_si_params_name" in cohort_key:
    del cohort_key["dlc_si_params_name"]
cohort_key["dlc_si_cohort_selection_name"] = "green_red_led"
cohort_key["bodyparts_params_dict"] = {
    "greenLED": si_params_name,
    "redLED_C": si_params_name,
}
print(cohort_key)

Here we'll insert the cohort into the `DLCSmoothInterpCohortSelection` table<br>..and populate `DLCSmoothInterpCohort`, which collates the separately smoothed and interpolated bodyparts into a single entry.

In [ ]:
sgp.DLCSmoothInterpCohortSelection().insert1(cohort_key, skip_duplicates=True)
sgp.DLCSmoothInterpCohort.populate(cohort_key)

And of course, let's make sure that the table populated correctly. 

In [ ]:
sgp.DLCSmoothInterpCohort.BodyPart() & cohort_key

#### [DLCCentroid](#TableOfContents) <a id='DLCCentroid1'></a>

We now have a cohort of smoothed and interpolated bodyparts from which to determine a centroid!<br>To start, we'll need a set of parameters to use for determining the centroid. For this tutorial, we can use the default.

In [ ]:
# Here is the default set
print(sgp.DLCCentroidParams.get_default())
centroid_params_name = "default"

>Here is the syntax to add your own parameters:
>```python
centroid_params = {
    'centroid_method': 'two_pt_centroid',
    'points' : {
        'point1': 'greenLED',
        'point2': 'redLED_C',},
    'speed_smoothing_std_dev': 0.100,
}
centroid_params_name = 'your_unique_param_name'
sgp.DLCCentroidParams.insert1({'dlc_centroid_params_name': centroid_params_name,
                                'params': centroid_params},
                                skip_duplicates=True)
```

And now let's make a key to insert into `DLCCentroidSelection`.

In [ ]:
centroid_key = cohort_key.copy()
fields = list(sgp.DLCCentroidSelection.fetch().dtype.fields.keys())
centroid_key = {key: val for key, val in centroid_key.items() if key in fields}
centroid_key["dlc_centroid_params_name"] = centroid_params_name
print(centroid_key)

Let's insert it into `DLCCentroidSelection` and then populate `DLCCentroid` !

In [ ]:
sgp.DLCCentroidSelection.insert1(centroid_key, skip_duplicates=True)
sgp.DLCCentroid.populate(centroid_key)

Here we can visualize the resulting centroid position

In [ ]:
(sgp.DLCCentroid() & centroid_key).fetch1_dataframe().plot.scatter(
    x="position_x",
    y="position_y",
    c="speed",
    colormap="viridis",
    alpha=0.5,
    s=0.5,
    figsize=(10, 10),
)

#### [DLCOrientation](#TableOfContents) <a id='DLCOrientation1'></a>

We'll now go through a similar process to identify the orientation!<br>To start, we'll need a set of parameters to use for determining the orientation. For this tutorial, we can use the default.

In [ ]:
print(sgp.DLCOrientationParams.get_default())
dlc_orientation_params_name = "default"

Here we'll prune the `cohort_key` we used above and add our `dlc_orientation_params_name` to make it suitable for `DLCOrientationSelection`.

In [ ]:
fields = list(sgp.DLCOrientationSelection.fetch().dtype.fields.keys())
orient_key = {key: val for key, val in cohort_key.items() if key in fields}
orient_key["dlc_orientation_params_name"] = dlc_orientation_params_name
print(orient_key)

And now let's insert into `DLCOrientationSelection` and populate `DLCOrientation` to determine the orientation!

In [ ]:
sgp.DLCOrientationSelection().insert1(orient_key, skip_duplicates=True)
sgp.DLCOrientation().populate(orient_key)

We can fetch the output of `DLCOrientation` as a dataframe to make sure everything looks appropriate.

In [ ]:
(sgp.DLCOrientation() & orient_key).fetch1_dataframe()

#### [DLCPosV1](#TableOfContents) <a id='DLCPosV1-1'></a>

Ok, we're now done with processing the position data! We just have to do some table manipulations to make sure everything ends up in the same format and same location.<br>
To summarize, we brought in a pretrained DLC project, used that model to run pose estimation on a new behavioral video, smoothed and interpolated the result, formed a cohort of bodyparts, and determined the centroid and orientation of this cohort. **_Whew!_**<br>
Now let's populate `DLCPosV1` with our centroid and orientation entries from above.<br>----<br>
To begin, we'll make a key that combines the cohort names we used for the orientation and centroid as well as the params names for both.

In [ ]:
fields = list(sgp.DLCPosV1.fetch().dtype.fields.keys())
dlc_key = {key: val for key, val in centroid_key.items() if key in fields}
dlc_key["dlc_si_cohort_centroid"] = centroid_key["dlc_si_cohort_selection_name"]
dlc_key["dlc_si_cohort_orientation"] = orient_key[
    "dlc_si_cohort_selection_name"
]
dlc_key["dlc_orientation_params_name"] = orient_key[
    "dlc_orientation_params_name"
]
print(dlc_key)

Now we can insert into `DLCPosSelection` and populate `DLCPos` with our `dlc_key`

In [ ]:
sgp.DLCPosSelection().insert1(dlc_key, skip_duplicates=True)
sgp.DLCPosV1().populate(dlc_key)

We can also make sure that all of our data made it through by fetching the dataframe attached to this entry.<br>We should expect 8 columns:
>time<br>video_frame_ind<br>position_x<br>position_y<br>orientation<br>velocity_x<br>velocity_y<br>speed

In [ ]:
(sgp.DLCPosV1() & dlc_key).fetch1_dataframe()

And even more, we can fetch the `pose_eval_result` that is calculated during this step. This field contains the percentage of frames that each bodypart was below the likelihood threshold of 0.95 as a means of assessing the quality of the pose estimation.

In [ ]:
(sgp.DLCPosV1() & dlc_key).fetch1("pose_eval_result")

#### [DLCPosVideo](#TableOfContents) <a id='DLCPosVideo1'></a>

Here we can create a video with the centroid and orientation overlaid on the animal's behavioral video. This will also plot the likelihood of each bodypart used in the cohort. This is completely optional, but a good idea to make sure everything looks correct.

In [ ]:
sgp.DLCPosVideoParams.insert_default()

In [ ]:
params = {
    "percent_frames": 0.05,
    "incl_likelihood": True,
}
sgp.DLCPosVideoParams.insert1(
    {"dlc_pos_video_params_name": "five_percent", "params": params},
    skip_duplicates=True,
)

In [ ]:
sgp.DLCPosVideoSelection.insert1(
    {**dlc_key, "dlc_pos_video_params_name": "five_percent"},
    skip_duplicates=True,
)

In [ ]:
sgp.DLCPosVideo().populate(dlc_key)

#### [PositionOutput](#TableOfContents) <a id='PositionOutput1'></a>

`PositionOutput` is the final table of the position pipeline and is automatically populated when we populate `DLCPosV1`! Let's make sure that our entry made it in.

In [ ]:
sgp.PositionOutput.merge_get_part(dlc_key)

`PositionOutput` also has a part table, similar to the `DLCModelSource` table above. Let's check that out as well.

In [ ]:
PositionOutput.DLCPosV1() & dlc_key

In [ ]:
merge_ids = PositionOutput.merge_get_part(dlc_key).fetch("merge_id")
(PositionOutput & {"merge_id": merge_ids[0]}).fetch1_dataframe()

#### [PositionVideo](#TableOfContents)<a id='PositionVideo1'></a>

Bonus points if you made it this far... We can use the `PositionVideo` table to create a video that overlays just the centroid and orientation (regardless of upstream source) on the behavioral video. This table uses the parameter `plot` to determine whether to plot the entry deriving from the DLC arm or from the Trodes arm of the position pipeline. This parameter also accepts 'all', which will plot both (if they exist) in order to compare results.

In [ ]:
sgp.PositionVideoSelection().insert1(
    {
        "nwb_file_name": "J1620210604_.nwb",
        "interval_list_name": "pos 13 valid times",
        "trodes_position_id": 0,
        "dlc_position_id": 1,
        "plot": "DLC",
        "output_dir": "/home/dgramling/Src/",
    }
)

In [ ]:
sgp.PositionVideo.populate({"plot": "DLC"})

### _CONGRATULATIONS!!_
Please treat yourself to a nice tea break :-)

### [Return To Table of Contents](#TableOfContents)<br>